<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Запити та парсинг

## 🔹 Що таке запит?

**Запит (HTTP request)** — це повідомлення, яке відправляє клієнт (браузер, Python-скрипт, мобільний застосунок) до сервера, щоб отримати або передати дані.  

🖥️ **Клієнт** — той, хто ініціює запит (наш код у Python, браузер).  
🌐 **Сервер** — той, хто обробляє запит і повертає відповідь.  

Цей механізм лежить в основі **вебу**: кожен раз, коли ми відкриваємо сторінку чи завантажуємо дані, відбувається серія запитів і відповідей.

---

## 🔹 Як виглядає HTTP-запит

У нього є кілька частин:

1. **Метод** (GET, POST, PUT, DELETE)

   * `GET` → отримати дані (найчастіше).
   * `POST` → відправити дані (логін, форма).
   * `PUT` → оновити.
   * `DELETE` → видалити.

2. **URL (Uniform Resource Locator)**

```
https://api.site.com:443/data?symbol=BTC&limit=10#section1
```

**Протокол** → `https`

   * Вказує, як передаються дані.
   * `http` – без шифрування.
   * `https` – із шифруванням (зараз стандарт).

**Домен (host)** → `api.site.com`

   * Основна адреса сайту.
   * Складається з:

     * `com` – домен верхнього рівня.
     * `site` – назва сайту.
     * `api` – піддомен (може бути `www`, `blog`, `shop` і т.д.).

**Порт (port)** → `:443` (часто прихований)

   * Номер "дверей", через які клієнт звертається до сервера.
   * `80` – HTTP за замовчуванням.
   * `443` – HTTPS за замовчуванням.

**Шлях (path)** → `/data`

   * Конкретний ресурс на сервері (сторінка, API-ендпоінт).

**Параметри (query string)** → `?symbol=BTC&limit=10`

   * Додають умови чи фільтри.
   * Формат: `ключ=значення` і розділяються `&`.
   * У прикладі:

     * `symbol=BTC` → отримати дані для Bitcoin.
     * `limit=10` → обмежити кількість результатів до 10.

**Фрагмент (anchor)** → `#section1`

   * Використовується у браузері для переходу до конкретної частини сторінки (сервер його не бачить).

3. **Headers (заголовки)**

   * "паспорт" нашого запиту:

     * `User-Agent` – браузер чи бот.
     * `Accept` – який формат даних ми хочемо (HTML/JSON).
     * `Authorization` – токен доступу.

4. **Body (тіло запиту)**

   * використовується у `POST/PUT` (наприклад, JSON із формою логіну).

---

## 🔹 Відповідь (HTTP response)

Сервер повертає:

* **status code** (200 – ок, 404 – не знайдено, 403 – заборонено).
* **headers** (тип контенту, cookies).
* **body** (дані: HTML, JSON, файл, картинка).

In [ ]:
import requests

url = "https://httpbin.org/get"
params = {"symbol": "BTC", "limit": 5}
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url, params=params, headers=headers)

print("URL після запиту:", res.url)
print("Код відповіді:", res.status_code)
print("Заголовки:", res.headers)
print("JSON:", res.json())


URL після запиту: https://httpbin.org/get?symbol=BTC&limit=5
Код відповіді: 200
Заголовки: {'Date': 'Mon, 08 Sep 2025 16:18:01 GMT', 'Content-Type': 'application/json', 'Content-Length': '367', 'Connection': 'keep-alive', 'Server': 'gunicorn/19.9.0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true'}
JSON: {'args': {'limit': '5', 'symbol': 'BTC'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Host': 'httpbin.org', 'User-Agent': 'Mozilla/5.0', 'X-Amzn-Trace-Id': 'Root=1-68bf01b7-336c0a84261c0718398bcb82'}, 'origin': '34.106.76.113', 'url': 'https://httpbin.org/get?symbol=BTC&limit=5'}



## 🔹 Етичні правила

`robots.txt` — це текстовий файл на сервері сайту, який містить правила для «роботів» (веб-скраперів, пошукових систем).
Наприклад, файл `https://example.com/robots.txt` може виглядати так:

```
User-agent: *
Disallow: /private/
Disallow: /admin/
Allow: /public/
```

**Пояснення:**

* `User-agent: *` → правило для всіх роботів.
* `Disallow: /private/` → ці сторінки не можна сканувати.
* `Allow: /public/` → ці сторінки можна сканувати.

Тобто сайт вказує, **які сторінки можна або не можна парсити**.

---

## 🔹 Як перевірити robots.txt

Просто відкрити URL у браузері:

```
https://example.com/robots.txt
```

Аналізуємо, що дозволено або заборонено.

---

## 🔹  Як програмно перевірити дозволені сторінки

Python має стандартний модуль `urllib.robotparser`

In [2]:
from urllib.robotparser import RobotFileParser

url = "https://example.com/products"
robots_url = "https://example.com/robots.txt"

rp = RobotFileParser()
rp.set_url(robots_url)
rp.read()

# Перевірка, чи дозволено доступ
if rp.can_fetch("*", url):
    print("Парсинг дозволено")
else:
    print("Парсинг заборонено")


Парсинг дозволено


**Пояснення коду:**

* `set_url()` — вказує, де знаходиться `robots.txt`;
* `read()` — зчитує правила;
* `can_fetch(user_agent, url)` — перевіряє, чи дозволено парсити конкретну сторінку для даного агента (`*` — будь-який агент).

---

## 🔹 Практичні правила етичного парсингу

1. **Не порушуйте `robots.txt`.**
2. **Не перевантажуйте сервер:** ставте затримки між запитами:

```python
import time
time.sleep(1)  # чекати 1 секунду між запитами
```

3. **Використовуйте API**, якщо сайт його надає — це швидше і безпечніше.
4. **Вказуйте user-agent** у запиті, щоб сайт знав, що це скрапер:

```python
headers = {"User-Agent": "MyDataBot/1.0 (+https://example.com)"}
requests.get(url, headers=headers)
```

5. **Не збирайте особисті дані користувачів**, це порушує законодавство.

**Task 1:**
1. Зробіть GET-запит до **[https://example.com/](https://example.com/)**.
   * Виведіть **статус-код** відповіді.
   * Подивіться перші 300 символів HTML (`.text[:300]`).
2. Змініть **User-Agent** у заголовках:
   * Спочатку відправте запит без User-Agent.
   * Потім з `"User-Agent": "MyCustomBot/1.0"`.
   * Порівняйте відповіді (чи змінився результат).

# **Основні HTML-теги для аналітика даних**

Ці теги найчастіше зустрічаються при парсингу сайту. З їх допомогою ми отримуємо текст, таблиці, списки, зображення та посилання.

---

## **1️⃣ Структурні теги**

| Тег         | Призначення                                     | Приклад                              |
| ----------- | ----------------------------------------------- | ------------------------------------ |
| `<div>`     | Блоковий контейнер для групування елементів     | `<div class="product">...</div>`     |
| `<span>`    | Рядковий контейнер для виділення частини тексту | `<span class="price">100 грн</span>` |
| `<section>` | Семантичний блок контенту                       | `<section>Новини</section>`          |
| `<article>` | Відокремлений блок (стаття, пост, оголошення)   | `<article><h2>Назва</h2></article>`  |

---

## **2️⃣ Текстові теги**

| Тег           | Призначення                                              | Приклад                   |
| ------------- | -------------------------------------------------------- | ------------------------- |
| `<p>`         | Абзац тексту — **основний для описів, статей, відгуків** | `<p>Опис продукту</p>`    |
| `<h1>`–`<h6>` | Заголовки різних рівнів (h1 — найважливіший)             | `<h2>Продукти</h2>`       |
| `<strong>`    | Жирний текст, виділення важливого                        | `<strong>Акція!</strong>` |
| `<em>`        | Курсивний текст, виділення                               | `<em>Новинка</em>`        |

---

## **3️⃣ Посилання та зображення**

| Тег     | Призначення                            | Приклад                                     |
| ------- | -------------------------------------- | ------------------------------------------- |
| `<a>`   | Посилання на інші сторінки або ресурси | `<a href="https://example.com">Перейти</a>` |
| `<img>` | Зображення                             | `<img src="img.jpg" alt="Фото продукту">`   |

---

## **4️⃣ Списки**

| Тег    | Призначення          | Приклад                     |
| ------ | -------------------- | --------------------------- |
| `<ul>` | Ненумерований список | `<ul><li>Пункт 1</li></ul>` |
| `<ol>` | Нумерований список   | `<ol><li>Пункт 1</li></ol>` |
| `<li>` | Елемент списку       | `<li>Пункт списку</li>`     |

---

## **5️⃣ Таблиці**

| Тег       | Призначення       | Приклад                         |
| --------- | ----------------- | ------------------------------- |
| `<table>` | Таблиця           | `<table>...</table>`            |
| `<tr>`    | Рядок таблиці     | `<tr><td>1</td><td>2</td></tr>` |
| `<td>`    | Клітинка даних    | `<td>Дані</td>`                 |
| `<th>`    | Заголовок колонки | `<th>Назва</th>`                |

---

## BeautifulSoup

`BeautifulSoup` – це бібліотека для **парсингу HTML і XML**.

### Основні методи:

* `soup.title`, `soup.a`, `soup.p` – доступ до тегів.
* `find(tag, attrs={})` – пошук першого елемента.
* `find_all(tag, attrs={})` – пошук усіх елементів.
* `select("css_selector")` – пошук по CSS-селекторах.
* `.text` / `.get("href")` – отримання тексту/атрибутів.

In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests

response = requests.get('https://example.com/')
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
print(response.text)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

### 1. Доступ до тегів напряму

* `soup.title` → перший тег `<title>`
* `soup.p` → перший тег `<p>`
* `soup.a` → перший тег `<a>`

In [ ]:
print("Заголовок:", soup.title.text)   # Example Domain
print("Абзац:", soup.p.text)           # This domain is for use in illustrative examples...
print("Посилання:", soup.a.text)       # More information...

Заголовок: Example Domain
Абзац: This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.
Посилання: More information...


### 2. `find(tag, attrs={})`

Знаходить **перший елемент**, який відповідає умовам.

In [ ]:
first_link = soup.find("a") # Особливість: якщо елемента немає, повертає `None`.
print(first_link.text)
print(first_link.get("href"))

More information...
https://www.iana.org/domains/example


### 3. `find_all(tag, attrs={})`

Повертає **список усіх елементів**.

In [ ]:
all_paragraphs = soup.find_all("p") # Особливість: повертає список, навіть якщо елемент один.
for p in all_paragraphs:
    print(p.text)

This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.
More information...


### 4. `select("css_selector")`

Дозволяє шукати елементи за **CSS-селекторами**, як у веб-розробці.

* `"p"` → усі абзаци
* `"a"` → усі посилання
* `"h1"` → заголовки першого рівня

In [ ]:
links = soup.select("a")
for link in links:
    print(link.text, "→", link.get("href"))

More information... → https://www.iana.org/domains/example


### 5. `.text` та `.get("атрибут")`

* `.text` → повертає **текст всередині тега**.
* `.get("href")` → повертає значення атрибуту (наприклад, посилання).

In [ ]:
title = soup.title.text
link_url = soup.a.get("href")

print("Заголовок сторінки:", title)
print("Посилання:", link_url)

Заголовок сторінки: Example Domain
Посилання: https://www.iana.org/domains/example


### Особливості роботи з BeautifulSoup

1. **Відсутні елементи** → якщо шукаєш тег, а його нема → буде `None`.

   ```python
   print(soup.find("h2"))  # None
   ```
2. **HTML може бути "кривим"** → але BeautifulSoup автоматично "підчистить".
3. **Методів багато** – найчастіше використовуються:

   * `.find()`
   * `.find_all()`
   * `.select()`
   * `.text`, `.get()`

## Парсинг реальних сайтів (CoinMarketCap)

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://coinmarketcap.com/"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

In [ ]:
table = soup.find("table", class_="cmc-table")

In [ ]:
print(table.prettify())

<table class="sc-7e3c705d-3 keBvNC cmc-table">
 <colgroup>
  <col style="width:50px;min-width:auto;max-width:auto"/>
  <col style="width:50px;min-width:auto;max-width:auto"/>
  <col style="width:250px;min-width:auto;max-width:auto"/>
  <col/>
  <col style="width:84px;min-width:auto;max-width:auto"/>
  <col style="width:84px;min-width:auto;max-width:auto"/>
  <col style="width:84px;min-width:auto;max-width:auto"/>
  <col style="width:200px;min-width:auto;max-width:auto"/>
  <col style="width:200px;min-width:auto;max-width:auto"/>
  <col style="width:200px;min-width:auto;max-width:auto"/>
  <col/>
 </colgroup>
 <thead>
  <tr>
   <th class="stickyTop">
   </th>
   <th class="stickyTop" style="text-align:start">
    <div class="sc-65e7f566-0 BmVAr">
     <span class="sc-65e7f566-0 iSzLgt rank-column-title">
      #
     </span>
    </div>
   </th>
   <th class="stickyTop" style="text-align:start">
    <div class="sc-e297ecb-0 bSCmGv sortable-header-container">
     <div class="sc-e297ecb-1

In [ ]:
print([col.text for col in table.thead.find_all("th")])

['', '#', 'Name', 'Price', '1h %', '24h %', '7d %', 'Market Cap', 'Volume(24h)', 'Circulating Supply', 'Last 7 Days']


In [ ]:
tbody = table.tbody.find_all('tr')


data = []
for item in tbody:
  try:
    cells = item.find_all('td')
    if not cells or len(cells) < 4:
      continue
    rank = cells[1].text.strip()
    name = cells[2].select_one('p.coin-item-name').text.strip()
    symbol = cells[2].select_one('p.coin-item-symbol').text.strip()
    price = cells[3].text.strip()

    data.append({
        'rank': rank,
        'name': name,
        'symbol': symbol,
        'price': price
        })
  except:
    continue

In [ ]:
data

[{'rank': '1', 'name': 'Bitcoin', 'symbol': 'BTC', 'price': '$112,395.42'},
 {'rank': '2', 'name': 'Ethereum', 'symbol': 'ETH', 'price': '$4,334.00'},
 {'rank': '3', 'name': 'XRP', 'symbol': 'XRP', 'price': '$2.97'},
 {'rank': '4', 'name': 'Tether', 'symbol': 'USDT', 'price': '$0.9999'},
 {'rank': '5', 'name': 'BNB', 'symbol': 'BNB', 'price': '$880.25'},
 {'rank': '6', 'name': 'Solana', 'symbol': 'SOL', 'price': '$216.48'},
 {'rank': '7', 'name': 'USDC', 'symbol': 'USDC', 'price': '$0.9998'},
 {'rank': '8', 'name': 'Dogecoin', 'symbol': 'DOGE', 'price': '$0.2392'},
 {'rank': '9', 'name': 'TRON', 'symbol': 'TRX', 'price': '$0.3338'},
 {'rank': '10', 'name': 'Cardano', 'symbol': 'ADA', 'price': '$0.8652'}]

## Автоматичний збір з кількох сторінок

In [ ]:
import time
import pandas as pd

i = 1
res = []
while i < 10:

  url = f'https://coinmarketcap.com/?page={i}'
  soup = BeautifulSoup(requests.get(url, headers=headers).text, 'html.parser')
  table = soup.find('table', class_='cmc-table')
  if not table:
    break

  tbody = table.tbody.find_all('tr')

  if not tbody:
    break

  tbody = table.tbody.find_all('tr')
  data = []

  for item in tbody:
    try:
      cells = item.find_all('td')
      if not cells or len(cells) < 4:
        continue
      rank = cells[1].text.strip()
      name = cells[2].select_one('p.coin-item-name').text.strip()
      symbol = cells[2].select_one('p.coin-item-symbol').text.strip()
      price = cells[3].text.strip()

      data.append({
          'rank': rank,
          'name': name,
          'symbol': symbol,
          'price': price
          })
    except:
      continue
  res += data
  time.sleep(3)
  i+=1

df = pd.DataFrame(res)
df

1
2
3
4
5
6
7
8
9


,rank,name,symbol,price
0,1,Bitcoin,BTC,"$112,564.36"
1,2,Ethereum,ETH,"$4,340.95"
2,3,XRP,XRP,$2.98
3,4,Tether,USDT,$0.9999
4,5,BNB,BNB,$878.82
...,...,...,...,...
85,806,Telos,TLOS,$0.05963
86,807,GUNZ,GUN,$0.02327
87,808,Groestlcoin,GRS,$0.2939
88,809,Portal,PORTAL,$0.04445


**Task 3:**

1. Зберіть перші 3 сторінки і порахуйте, скільки монет зібрали.
2. Збережіть у CSV.
3. Зробіть графік цін топ-20 монет.

## Альтернативний підхід: `pandas.read_html`

✅ **`pd.read_html(url)`**

* Шукає всі `<table>` у HTML.
* Повертає **список DataFrame**, де кожна таблиця — окремий DataFrame.
* Якщо таблиць кілька, доступ до них відбувається так: `tables[0]`, `tables[1]`, … ([Pandas][2])

📌 **Корисні параметри:**

| Параметр                | Призначення                                                |               |
| ----------------------- | ---------------------------------------------------------- | ------------- |
| `match="текст"`         | Вибрати таблиці, де є заданий текст у заголовку або вмісті |               |
| `attrs={'id': 'value'}` | Вибрати таблиці з конкретними атрибутами HTML              |               |
| `header=0`              | Використовувати перший рядок як заголовки колонок          |               |
| `index_col=0`           | Вказати колонку як індекс                                  |               |
| `parse_dates=[...]`     | Перетворити деякі колонки у datetime                       |

Якщо таблиця некоректно зчитується — можна спочатку використати BeautifulSoup, витягти HTML і передати в pd.read_html(str(soup)).

Для великих сторінок або декількох таблиць — ітеруйте по списку, щоб знайти потрібну.

Завжди перевіряйте df.head() та df.info() після зчитування, щоб упевнитися, що дані завантажились правильно.

In [3]:
import pandas as pd

url = "https://scrapeme.live/population-data.html"

# Зчитати всі таблиці на сторінці
tables = pd.read_html(url)

# Перевіряємо, скільки таблиць знайшов Pandas
print("Кількість таблиць:", len(tables))

# Перша таблиця
df1 = tables[0]
print("Перші 5 рядків першої таблиці:")
print(df1.head())

# Друга таблиця
df2 = tables[1]
print("Перші 5 рядків другої таблиці:")
print(df2.head())


Кількість таблиць: 2
Перші 5 рядків першої таблиці:
  Rank     Dependency  Population % of the world          Date  \
0    –          World  7989280000           100%   4 Nov  2022   
1    1          China  1412600000          17.7%  31 Dec  2021   
2    2          India  1375586000          17.2%   1 Mar  2022   
3    3  United States   331893745          4.15%   1 Jul  2021   
4    4      Indonesia   275773800          3.45%   1 Jul  2022   

                Source                                              Notes  
0        UN projection                                                NaN  
1    Official estimate  The population figure refers to mainland China...  
2  Official projection  The figure includes the population of the Indi...  
3    Official estimate  The figure includes the 50 states and  the Dis...  
4    Official estimate                                                NaN  
Перші 5 рядків другої таблиці:
  hide  v  t  e  Lists of countries by population statistics  \


## Обмеження і ризики

1. **Етика** – перевіряти `robots.txt`.
2. **Захист** – капчі, блокування.
3. **User-Agent** – треба маскувати під браузер.
4. **Частота запитів** – обмежувати `time.sleep`.
5. **API завжди краще** ніж HTML-парсинг.

## Додаткові завдання для практики

**Task 1.** Зробіть парсинг заголовків новин з **[https://news.ycombinator.com/](https://news.ycombinator.com/)**.  
**Task 2.** Витягніть курс валют з **[https://www.x-rates.com/](https://www.x-rates.com/)**.  